In [1]:
import os
import sys

sys.path.insert(0, f'{os.environ.get("HOME")}/workspace/recommendation-study')

In [2]:
from collections import defaultdict
import numpy as np
from util.models import Dataset, RecommendResult
from base import BaseRecommender

np.random.seed(0)

In [3]:
class RandomRecommender(BaseRecommender):
    def recommend(self, dataset: Dataset, k: int, **kwargs) -> RecommendResult:
        unique_user_ids = sorted(dataset.test.user_id.unique())
        unique_movie_ids = sorted(dataset.test.movie_id.unique())

        user_id2index = {
            user_id: index for index, user_id in enumerate(unique_user_ids)
        }

        movie_id2index = {
            movie_id: index for index, movie_id in enumerate(unique_movie_ids)
        }

        pred_matrix = np.random.uniform(0.5, 5.0, (len(unique_user_ids), len(unique_movie_ids)))

        pred_results = []
        for _, row in dataset.test.iterrows():
            user_id = row['user_id']
            movie_id = row['movie_id']

            user_index = user_id2index[user_id]
            movie_index = movie_id2index[movie_id]

            pred_score = pred_matrix[user_index, movie_index]
            pred_results.append(pred_score)

        movie_rating_predict = dataset.test.copy()
        movie_rating_predict['rating_pred'] = pred_results

        pred_user2items = defaultdict(list)

        for user_id in unique_user_ids:
            user_index = user_id2index[user_id]
            movie_indices = np.argsort(-pred_matrix[user_index, :])
            for movie_index in movie_indices:
                movie_id = unique_movie_ids[movie_index]
                pred_user2items[user_id].append(movie_id)
                if len(pred_user2items[user_id]) >= k:
                    break

        return RecommendResult(movie_rating_predict.rating_pred, pred_user2items)

In [4]:
RandomRecommender().run_sample()

rmse: 1.901, precision@K: 0.002, recall@K: 0.005
